# Main code to execute

In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [ ]:
import vision as vis
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion



COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [ ]:
# Main -> state machine
state = COMPUTE
obstacle = False
path_calculated = False

cap = cv.VideoCapture(2)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos, radius_pxl = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    scale_factor = pos.convert_meter2pxl(radius_pxl)
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        for subgoal in path_arch[1:]:
            robot_pos_travel = robot_pos.copy()
            while not motion.check_robot_arrived(robot_pos_travel,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                last_pos = robot_pos_travel.copy()
                success, robot_pos_travel = pos.get_robot_position(cropped_travel)
                if not success:
                    robot_pos_travel = last_pos.copy()
                    print("thymio disappeared",robot_pos_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos_travel,im4)
                vis.annotate_path(path_arch,im4)
                print("robot angle", robot_pos[1])
                err = motion.get_error(robot_pos_travel,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(1)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        for subgoal in path_goal:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                last_pos = robot_pos
                success, robot_pos = pos.get_robot_position(cropped_travel)
                if not success:
                    robot_pos = last_pos
                    print("thymio disappeared")
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_goal,im4)
                print("robot angle", robot_pos[1])
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(1)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    #cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    

---

# Global navigation

**For simplicity, the lists' indices used in the Global Navigation module follow this convention: <br>
index 0 corresponds to start, maximal index corresponds to goal, and all intermediary indices correspond to obstacle vertices. <br>
This allows us to work with points' indices (also referred to as names) rather than points' coordinates.**
<br/>

## Visibility graph
The code for computing the visibility graph is given in global_path.py.<br/>

It takes as input:
- start, the start position as tuple (x,y)
- goal, the goal position as tuple (x,y)
- list_vertices, the position of every obstacle's vertices grouped by obstacle: <br>
[[(x1.1,y1.1),(x1.2,y1.2),...,(x1.V1,y1.V1)],...,[(xO.1,yO.1),(xO.2,yO.2),...,(xO.VO,yO.VO)]] <br>
where Vi is the number of vertices of obstacle i, and O is the total number of obstacles. <br/>

The output is list_neighbours = <br>
[[neighbours_of_start], [neighbours_of_vertex1],..., [neighbours_of_goal]] <br>
where neighbour_of_XXX is list of tuples which each contain the index and distance of neighbours: (index_neighbour, distance_neighbour)

<br/>

### Initialization

<br/>
The initialization consists in two steps :<br>

1. Listing all points on the graph. <br>
In this step, the coordinates of every obstacle's vertices, as well as start's and goal's coordinates, are read and stored together in a list. <br>
Also, all coordinates are transformed into instances of class check_intersection.Point. This will allow us to use the function check_intersection.doIntersect() more easily.<br>
*The code check_intersection.py was taken from https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/ (cons. 27.11.2021) and was developed by Ansh Riyal.*


2. Listing all obstacle sides.<br>
In this step, we want to construct each obstacle as a series of vertex indices. For example, a triangle could be stored as [(1,2),(2,3), (3,1)]. <br>
To do so, we start by counting how many vertices (i.e. how many sides) each obstacle has. <br>
Then, for each obstacle, we build a list of its sides. To do so, we first need to calculate a shift k from which to start giving vertices' indices. For example, for an obstacle whose first vertex's name is k and which has v vertices, the result would be: [(k,k+1), (k+1,k+2), ..., (k+(v-1), k)]. <br>
Finally, we store all lists of sides into a bigger list.

<br/>
The codes for initialization are given in the following cells.

In [ ]:
def flatten_list_points(list_vertices, start, goal):
    """
    Returns a "flat" (i.e. list without sublists or tuples) version of
    all point coordinates including start and goal.
    Each entry is an instance of class check_intersection.Point
    """    
    list_points = []         
    for obstacle in range(len(list_vertices)):
        for vertex in range(len(list_vertices[obstacle])):
            x = list_vertices[obstacle][vertex][0]
            y = list_vertices[obstacle][vertex][1]
            list_points.append(check_intersection.Point(x,y))                   #insert obstacle vertices
                                                            
    list_points.insert(0, check_intersection.Point(start[0],start[1]))          #insert start
    list_points.append(check_intersection.Point(goal[0],goal[1]))               #insert goal
    
    return list_points

In [ ]:
def obtain_list_sides(list_vertices):
    """
    Given the list of obstacle vertices sorted by obstacle,
    this function returns the list of obstacle sides sorted by obstacle.
    """   
    #step 1: obtain the number of sides of each obstacle
    length_obstacles = [len(obstacle) for obstacle in list_vertices]
        
    #step 2: convert indexing of vertices from [0][0],[0][1],... into 1,2,3,...,N 
    #by introducing a shift (0 and N+1 are start and goal respectively)
    
    #then enter every tuple (side.vertex1, side.vertex2) into sides_obstacle
    #then enter sides_obstacle into list_sides
    list_sides = []
    for obstacle in range(len(list_vertices)):
        sides_obstacle = []                             #list of sides of current obstacle
        
        shift = sum(length_obstacles[0:obstacle]) + 1   #to convert indexing
        nb_sides = length_obstacles[obstacle]
        for vertex in range(nb_sides):
            side = (shift + vertex,\
                    shift + ((vertex+1) % nb_sides))    #use of % to count 1,2,1 instead of 1,2,3
            sides_obstacle.append(side)
        
        list_sides.append(sides_obstacle)               #total list of sides, grouped by obstacle

    return list_sides

### Completing the list of neighbours
<br/>

We now want to indicate for each point:
- who its neighbours are
- how far they are from that point<br>

*(Note that the term "neighbour" refers to all points that are visible by the current point, i.e. no line connecting two neighbours passes trough an obstacle.)*
<br/><br/>

To simplify our search, we assume that our obstacles are simple shapes such as squares or triangles. Thanks to this assumption, we only have five different possibilities to determine whether two points p1,p2 are neighbours (their connection line **CL** is valid) or not: <br>

1. (p1,p2) is a side of an obstacle $\rightarrow$ CL is valid (segment v1 on the following figure)<br>
2. p1 and p2 lie on the same obstacle, but CL is not a side of the obstacle $\rightarrow$ CL is not valid (segment nv1 on the following figure)
3. p1 and p2 do not lie on the same obstacle. p1 and/or p2 lie on all lines that CL intersects $\rightarrow$ CL is valid (segment v2 on the following figure)
4. p1 and p2 do not lie on the same obstacle. CL intersects some lines on which neither p1 nor p2 lie $\rightarrow$ CL is not valid (segment nv2 on the following figure)
5. Other cases: we consider that CL is valid (this did not cause any problems upon testing).

Note that this logic is not robust to complicated obstacle shapes. On the following figure, nv3 would not be valid due to condition 2. However, this logic is sufficient with the kind of obstacles that we use.



![Visibility](./images/VisibilityGraph.JPG) <br/>

Note the indexing convention: point 0 (also noted as A) is the start, point 15 (also noted as B) is the goal, all intermediary indices are obstacles' vertices. <br/>

Our algorithm considers that v1 and v2 are valid. <br>
nv1,nv2,nv3 are not valid.

Now that these conditions are set, here is the pseudo-code to explain the logic on one point and one obstacle:

And now, the python implementation for all points and obstacles: 

In [ ]:
def find_all_paths(list_vertices, start, goal):
    # 1. Initialisation
    list_points = flatten_list_points(list_vertices, start, goal)
    N = len(list_points) - 2    #number of obstacle vertices
    
    list_sides = obtain_list_sides(list_vertices) 

    # 2. Search algorithm
    # Determine for each point, what other points it can "see" 
    # 5 conditions as listed in the report
    dist = lambda x1,y1,x2,y2: math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    list_neighbours = []                        #final output: list of neighbours of all points
    for point1_name in range(N+2):
        ngb_current_point = []                  #list of neighbours of the current point
        
        for point2_name in range(N+2):
            if point1_name==point2_name:        #do not check self-connection
                pass
            
            
            else:
                point1 = list_points[point1_name]
                point2 = list_points[point2_name]
                valid_connection = True         #valid until proven wrong
                
                #Test the 5 conditions listed above
                for obstacle in list_sides:
                    obstacle_flattened = [item for sublist in obstacle for item in sublist]
                    
                    
                    condition1 = (((point1_name,point2_name) in obstacle)\
                                  or ((point2_name,point1_name) in obstacle))
                        
                    condition2 = ((point1_name in obstacle_flattened)\
                                  and (point2_name in obstacle_flattened))
                    
                    if condition1:     
                        pass
                        
                    elif condition2:        
                        valid_connection = False                        
                        
                    else:
                        for side_vertices in obstacle:
                            vertex1_name = side_vertices[0]
                            vertex2_name = side_vertices[1]
                            vertex1 = list_points[vertex1_name]
                            vertex2 = list_points[vertex2_name]
                            
                            
                            condition3 = ((point1_name==vertex1_name) or (point2_name==vertex2_name)\
                                or (point2_name==vertex1_name) or (point1_name==vertex2_name))
                            
                            condition4 = check_intersection.doIntersect(point1, point2, vertex1, vertex2)
                                
                            
                            if condition3:         
                                    pass
                                    
                            elif condition4:          
                                    valid_connection = False
                            
                            #else: condition5=True ; pass
                
                if valid_connection:
                    ngb_current_point.append((point2_name, dist(point1.x, point1.y, point2.x, point2.y)))
        
        list_neighbours.append(ngb_current_point)
        
    return list_neighbours
